In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd

In [3]:
dict_storage_options = {'endpoint_url':'http://localhost:9000',
                        'key':'minioadmin',
                        'secret':12345678}

# Change date to current date
parquet_df = pd.read_parquet('s3://tiki/curated/dim_product/ingestion_date=2024-04-01', storage_options=dict_storage_options) 
parquet_df.head(2)

,tiki_pid,name,brand_name,origin,ingestion_dt_unix
0,270851667,Mặt nạ giấy dưỡng da Hàn Quốc innisfree My Rea...,Innisfree,Hàn Quốc,1711993659
1,252608882,Tã/bỉm quần HUGGIES SKINCARE MEGA JUMBO size X...,Huggies,Việt Nam,1711993659


In [7]:

# Find the row with the maximum 'ingestion_dt_unix'
latest_row = parquet_df.loc[parquet_df['ingestion_dt_unix'].idxmax()]

# Filter DataFrame to get the latest data
latest_parquet = parquet_df[parquet_df['ingestion_dt_unix'] == latest_row['ingestion_dt_unix']]

# Define PostgreSQL connection parameters
postgres_username = 'my_user'
postgres_password = 'my_password'
postgres_host = 'localhost'
postgres_port = '35432'
postgres_db = 'dw_tiki'

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{postgres_username}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}')

# Define the table name
table_name = 'dim_product'

# Write the latest data to PostgreSQL
latest_parquet.to_sql(table_name, engine, if_exists='replace', index=False)

# Display the latest data
print(latest_parquet)


      tiki_pid                                               name brand_name  \
0    270851667  Mặt nạ giấy dưỡng da Hàn Quốc innisfree My Rea...  Innisfree   
1    252608882  Tã/bỉm quần HUGGIES SKINCARE MEGA JUMBO size X...    Huggies   
2    252608891  Tã/bỉm quần HUGGIES SKINCARE gói SUPER JUMBO s...    Huggies   
3    210966404  Combo Sữa dưỡng thể Vaseline Gluta-Hya Dưỡng D...   Vaseline   
4    203853390                    Điện Thoại Oppo A17k (3GB/64GB)       OPPO   
..         ...                                                ...        ...   
151    1453915  Ly Giữ Nhiệt Bằng Thép Không Gỉ Lock&Lock Clip...  LocknLock   
152    1672157  Bao cao su Durex Invisible Extra Thin Extra Se...      Durex   
153     897016  Combo Sân Chơi Trí Tuệ Của Chim Đa Đa (Bộ 6 Cuốn)       None   
154     579962  Bộ Nồi 5 Đáy Cao Cấp Núm Vàng Sunhouse SH781 Q...   Sunhouse   
155     414764  Lò Vi Sóng Sharp R-205VN(S) - 20L - Hàng chính...      Sharp   

         origin  ingestion_dt_unix  
0 